## 1. Subscribe to the model package

To subscribe to the model package:

1. Open the model package listing page [Deidentify Clinical Documents (EN)](https://aws.amazon.com/marketplace/pp/prodview-ept2dbql5slue)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Deidentify Clinical Document (EN)

- **Model**: `clinical_deidentification_multi_mode_output_en`

- **Model Description**: This pipeline de-identifies PHI information from medical texts by masking and obfuscating entities such as `AGE`, `CONTACT`, `DATE`, `LOCATION`, `NAME`, `PROFESSION`, `CITY`, `COUNTRY`, `DOCTOR`, `HOSPITAL`, `IDNUM`, `MEDICALRECORD`, `ORGANIZATION`, `PATIENT`, `PHONE`, `EMAIL`, `STREET`, `USERNAME`, `ZIP`, `ACCOUNT`, `LICENSE`, `VIN`, `SSN`, `DLN`, `PLATE`, `IPADDR`, producing masked with entity labels, fixed-length char, same-length char, and obfuscated versions of the text simultaneously.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "clinical-deidentification-multi-mode-output-en"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

---------!

Once endpoint has been created, you would be able to perform real-time inference.

In [6]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [7]:
docs = [
'''Mr. William Garcia, 25 years-old , born in Beijing, was transfered to the The Johns Hopkins Hospital. Phone number: (541) 754-3010. MSW: 100009632582 . for his colonic polyps.  William Garcia wants to know the results from them.  He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week.  William Garcia has cut back his cigarettes to one time per week. MR. # 50712249 P:   Follow up with Dr. Hobbs Spruce in 3 months. Gilbert P. Perez, M.D. Johns Hopkins Hospital 1800 Orleans St. PHONE: (+1) 410-955-5000 ''',

'''Record date : 2022-01-13 , Ethan Hale , M.D . , Name : Ava Davis. EMAIL: davis_a@gmail.com. MR # 7190334  Phone : (555) 555-1212. Date : 01-13-1993 PCP : Oliveira , 29 years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities.  Ava Davis was noted to have a large sacral wound; this is in a similar location with his previous laminectomy, and this continues to receive daily care. DD: 01/13/2022 . DV: 01/13/2022. Ethan Hale , M.D. MSW 754443200936. NewYork-Presbyterian Hospital . 525 East 68th Street. (85) 555-1212. ''',

]


sample_text = """Record date : 2013-01-13, David Hale, M.D. is manager,  Name: Elvis Presley. Age: 17. Phone: (9) 7765-5632. MR. # 7194334 Date: 01-13-1993 PCP: Oliveira. Record date: 2012-11-09. Cocke County Baptist Hospital 0295 Keats Street. This 17-yr-old male, presented with chest heaviness that started during a pick-up basketball game. His past medical history was unremarkable. Elvis Presley denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age 38. Elvis Presley was a nonsmoker, did not drink alcohol, and denied recreational drug use. Elvis Presley swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal. DD: 01/13/2013 . DV: 01/13/2013, Cocke County Baptist Hospital 0295 Keats Street. PHONE : (+1) 423-625-2200 """

### Important Parameters

- **masking_policy**: `str`

    Users can select a masking policy to determine how sensitive entities are handled:

    - **masked**: Default policy that masks entities with their type.

      Example: "My name is Mike. I was admitted to the hospital yesterday."  
      -> "My name is `<PATIENT>`. I was admitted to the hospital yesterday."

    - **obfuscated**: Replaces sensitive entities with random values of the same type.

      Example: "My name is Mike. I was admitted to the hospital yesterday."  
      -> "My name is `Barbaraann Share`. I was admitted to the hospital yesterday."

    - **masked_fixed_length_chars**: Masks entities with a fixed length of asterisks (*).

      Example: "Name: Hendrickson, Ora, Record date: 2093-01-13, # 719435. Dr. John Green, E-MAIL: green@gmail.com."  
      -> "Name: `****`, Record date: `****`, # `****`. Dr. `****`, E-MAIL: `****`."

    - **masked_with_chars**: Masks entities with asterisks (*).

      Example: "Name: Hendrickson, Ora, Record date: 2093-01-13, # 719435. Dr. John Green, E-MAIL: green@gmail.com."  
      -> "Name: `[**************]`, Record date: `[********]`, # `[****]`. Dr. `[********]`, E-MAIL: `[*************]`."

- **sep**: `str`

    Separator used to join subparts within each prediction.

    By default, the separator is set to a single space (" "), but users can specify any other separator as needed. Necessary because the model outputs predictions as separate subparts, and the chosen separator is used to join them into coherent text.

    The separator must be one of the following characters: space (' '), newline ('\n'), comma (','), tab ('\t'), or colon (':').
    
You can specify these parameters in the input as follows:

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "masking_policy": "masked",
    "sep": " ",
}
```


### JSON

1. **Input format**: Single Text Document

Provide a single text document as a string.

  
  
```json
{
    "text": "Single text document"
}
```

2. **Input format**: Array of Text Documents

Use an array containing multiple text documents. Each element represents a separate text document.

```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

#### Example 1: masked (default-policy)

In [8]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"Record date : <DATE>, <DOCTOR>, M.D. is <PROFESSION>, Name: <PATIENT>. Age: <AGE>. Phone: <PHONE>. MR. # <MEDICALRECORD> Date: <DATE> PCP: <DOCTOR>. Record date: <DATE>. <HOSPITAL> <STREET>. This <AGE> male, presented with chest heaviness that started during a pick-up basketball game. His past medical history was unremarkable. <HOSPITAL> denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age <AGE>. <PATIENT> was a nonsmoker, did not drink alcohol, and denied recreational drug use. Elvis Presley swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal. DD: <DATE> . DV: <DATE>, <HOSPITAL> <STREET>. PHONE : <PHONE>"


#### Example 2: obfuscated

In [9]:
input_json_data = {"text": sample_text, "masking_policy": "obfuscated"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"Record date : 2013-02-02, Darliss Ek, M.D. is Insurance risk surveyor, Name: Charma Coon. Age: 18. Phone: (0) 8657-8469. MR. # 6295284 Date: 02-02-1993 PCP: Criss Dom. Record date: 2012-11-29. ALASKA REGIONAL HOSPITAL 2210 Troy Schenectady Rd. This 15-yr-old male, presented with chest heaviness that started during a pick-up basketball game. His past medical history was unremarkable. Charma Coon denied prior cardiac symptoms and suffered no chest trauma during the game. His father had suffered an acute myocardial infarction at age 36. Charma Coon was a nonsmoker, did not drink alcohol, and denied recreational drug use. Elvis Presley swallowed a tablet of aspirin before coming to the emergency room. His blood pressure was 160/90 mm Hg, and his heart rate was 80 bpm. Physical examination revealed no stigmata of Marfan syndrome. The rest of his physical examination was normal. DD: 02/02/2013 . DV: 02/02/2013, ALASKA REGIONAL HOSPITAL 2210 Troy Schenectady Rd. PHONE : (+1) 324-401-0272"


#### Example 3: masked (default-policy)

In [10]:
input_json_data = {"text": docs}
# The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.
data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"Mr. <PATIENT>, <AGE> years-old , born in <CITY>, was transfered to the The <HOSPITAL>. Phone number: <PHONE>. MSW: <MEDICALRECORD> . for his colonic polyps. <DOCTOR> wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week. <PATIENT> has cut back his cigarettes to one time per week. MR. # <MEDICALRECORD> P: Follow up with Dr. <DOCTOR> in 3 months. <DOCTOR> P. <DOCTOR>, M.D. <LOCATION>. PHONE: <PHONE>"
1,"Record date : <DATE> , <DOCTOR> , M.D . , Name : <PATIENT>. EMAIL: <EMAIL>. MR # <MEDICALRECORD> Phone : <PHONE>. Date : <DATE> PCP : <DOCTOR> , <AGE> years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. <NAME> was noted to have a large sacral wound; this is in a similar location with his previous laminectomy, and this continues to receive daily care. DD: <DATE> . DV: <DATE>. <DOCTOR> , M.D. MSW <PHONE>. <LOCATION>. <PHONE>."


#### Example 4: obfuscated

In [11]:
input_json_data = {"text": docs, "masking_policy": "obfuscated"}
# The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.
data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )

pd.DataFrame(data)

,predictions
0,"Mr. Kristi Petties, 39 years-old , born in Ubide, was transfered to the The BLUE RIDGE REGIONAL HOSPITAL, INC. Phone number: (106) 269-4854. MSW: 627035009381 . for his colonic polyps. Kristi Petties wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week. Kristi Petties has cut back his cigarettes to one time per week. MR. # 82993716 P: Follow up with Dr. Artelia Laroche in 3 months. Jesusita Morris P. Media Spikes, M.D. 2525 Court Drive. PHONE: (+9) 678-938-1017"
1,"Record date : 2022-02-02 , Argyle Kurk , M.D . , Name : Real Camp. EMAIL: Bradly@google.com. MR # 5102585 Phone : (277) 824-2353. Date : 02-02-1993 PCP : Criss Dom , 28 years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. Real Camp was noted to have a large sacral wound; this is in a similar location with his previous laminectomy, and this continues to receive daily care. DD: 02/02/2022 . DV: 02/02/2022. Argyle Kurk , M.D. MSW 614431540086. 22 S Greene St. (76) 195-0932."


### JSON Lines

1. **Input format**: Provide input in JSON Lines format, where each line is a JSON object representing a text document along with any optional parameters.

```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [12]:
import json

def create_jsonl(records, masking_policy):
    json_records = []

    for text in records:
        record = {
            "text": text,
            "masking_policy": masking_policy
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines


#### Example 1: masked (default-policy)

In [13]:
input_jsonl_data = create_jsonl(docs, masking_policy="masked")
# The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.
data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "Mr. <PATIENT>, <AGE> years-old , born in <CITY>, was transfered to the The <HOSPITAL>. Phone number: <PHONE>. MSW: <MEDICALRECORD> . for his colonic polyps. <DOCTOR> wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week. <PATIENT> has cut back his cigarettes to one time per week. MR. # <MEDICALRECORD> P:   Follow up with Dr. <DOCTOR> in 3 months. <DOCTOR> P. <DOCTOR>, M.D.  <LOCATION>. PHONE: <PHONE>"}
{"predictions": "Record date : <DATE> , <DOCTOR> , M.D . , Name : <PATIENT>. EMAIL: <EMAIL>. MR # <MEDICALRECORD>  Phone : <PHONE>. Date : <DATE> PCP : <DOCTOR> , <AGE> years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. <

#### Example 2: obfuscated

In [14]:
input_jsonl_data = create_jsonl(docs, masking_policy="obfuscated")
# The timeout for InvokeEndpoint is 60 seconds, so please carefully select the number of input texts you are sending in one request.
data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )

print(data)

{"predictions": "Mr. Kristi Petties, 39 years-old , born in Ubide, was transfered to the The BLUE RIDGE REGIONAL HOSPITAL, INC. Phone number: (149) 702-6378. MSW: 588502774128 . for his colonic polyps. Kristi Petties wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week. Kristi Petties has cut back his cigarettes to one time per week. MR. # 78676720 P:   Follow up with Dr. Artelia Laroche in 3 months. Jesusita Morris P. Media Spikes, M.D.  2525 Court Drive. PHONE: (+9) 470-962-8366"}
{"predictions": "Record date : 2022-02-02 , Argyle Kurk , M.D . , Name : Real Camp. EMAIL: Bradly@google.com. MR # 2947654  Phone : (650) 354-6568. Date : 02-02-1993 PCP : Criss Dom , 28 years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted t

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [16]:
import json
import os


input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)


validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

input_json_data = {
    "input1.json": {"text": docs, "masking_policy": "masked"},
    "input2.json": {"text": docs, "masking_policy": "obfuscated"},
    "input3.json": {"text": docs, "masking_policy": "masked_fixed_length_chars"},
    "input4.json": {"text": docs, "masking_policy": "masked_with_chars"}
}

input_jsonl_data = {
    "input1.jsonl": create_jsonl(docs, masking_policy="masked"),
    "input2.jsonl": create_jsonl(docs, masking_policy="obfuscated"),
    "input3.jsonl": create_jsonl(docs, masking_policy="masked_fixed_length_chars"),
    "input4.jsonl": create_jsonl(docs, masking_policy="masked_with_chars")
}


for file_name, data in input_json_data.items():
    write_and_upload_to_s3(data, f"{json_input_dir}/{file_name}")

for file_name, data in input_jsonl_data.items():
    write_and_upload_to_s3(data, f"{jsonl_input_dir}/{file_name}")



### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [18]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):
    
    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [19]:
masking_policies = {
    "masked": "input1.json",
    "obfuscated": "input2.json",
    "masked_fixed_length_chars": "input3.json",
    "masked_with_chars": "input4.json",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_json_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked


,predictions
0,"Mr. <PATIENT>, <AGE> years-old , born in <CITY>, was transfered to the The <HOSPITAL>. Phone number: <PHONE>. MSW: <MEDICALRECORD> . for his colonic polyps. <DOCTOR> wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week. <PATIENT> has cut back his cigarettes to one time per week. MR. # <MEDICALRECORD> P: Follow up with Dr. <DOCTOR> in 3 months. <DOCTOR> P. <DOCTOR>, M.D. <LOCATION>. PHONE: <PHONE>"
1,"Record date : <DATE> , <DOCTOR> , M.D . , Name : <PATIENT>. EMAIL: <EMAIL>. MR # <MEDICALRECORD> Phone : <PHONE>. Date : <DATE> PCP : <DOCTOR> , <AGE> years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. <NAME> was noted to have a large sacral wound; this is in a similar location with his previous laminectomy, and this continues to receive daily care. DD: <DATE> . DV: <DATE>. <DOCTOR> , M.D. MSW <PHONE>. <LOCATION>. <PHONE>."




Masking policy: obfuscated


,predictions
0,"Mr. Kristi Petties, 39 years-old , born in Ubide, was transfered to the The BLUE RIDGE REGIONAL HOSPITAL, INC. Phone number: (951) 884-1660. MSW: 630160109323 . for his colonic polyps. Kristi Petties wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week. Kristi Petties has cut back his cigarettes to one time per week. MR. # 55732202 P: Follow up with Dr. Artelia Laroche in 3 months. Jesusita Morris P. Media Spikes, M.D. 2525 Court Drive. PHONE: (+5) 427-062-3762"
1,"Record date : 2022-02-02 , Argyle Kurk , M.D . , Name : Real Camp. EMAIL: Bradly@google.com. MR # 8315176 Phone : (160) 737-1062. Date : 02-02-1993 PCP : Criss Dom , 28 years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. Real Camp was noted to have a large sacral wound; this is in a similar location with his previous laminectomy, and this continues to receive daily care. DD: 02/02/2022 . DV: 02/02/2022. Argyle Kurk , M.D. MSW 694854627035. 22 S Greene St. (00) 938-1829."




Masking policy: masked_fixed_length_chars


,predictions
0,"Mr. ****, **** years-old , born in ****, was transfered to the The ****. Phone number: ****. MSW: **** . for his colonic polyps. **** wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week. **** has cut back his cigarettes to one time per week. MR. # **** P: Follow up with Dr. **** in 3 months. **** P. ****, M.D. ****. PHONE: ****"
1,"Record date : **** , **** , M.D . , Name : ****. EMAIL: ****. MR # **** Phone : ****. Date : **** PCP : **** , **** years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. **** was noted to have a large sacral wound; this is in a similar location with his previous laminectomy, and this continues to receive daily care. DD: **** . DV: ****. **** , M.D. MSW ****. ****. ****."




Masking policy: masked_with_chars


,predictions
0,"Mr. [************], ** years-old , born in [*****], was transfered to the The [********************]. Phone number: [************]. MSW: [**********] . for his colonic polyps. [************] wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week. [************] has cut back his cigarettes to one time per week. MR. # [******] P: Follow up with Dr. [**********] in 3 months. [*****] P. [***], M.D. [************************************]. PHONE: [***************]"
1,"Record date : [********] , [********] , M.D . , Name : [*******]. EMAIL: [***************]. MR # [*****] Phone : [************]. Date : [********] PCP : [******] , ** years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegia, lower extremities. [*******] was noted to have a large sacral wound; this is in a similar location with his previous laminectomy, and this continues to receive daily care. DD: [********] . DV: [********]. [********] , M.D. MSW [**********]. [**************************************************]. [***********]."


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [21]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [22]:
masking_policies = {
    "masked": "input1.jsonl",
    "obfuscated": "input2.jsonl",
    "masked_fixed_length_chars": "input3.jsonl",
    "masked_with_chars": "input4.jsonl",
}

for policy_name, validation_file_name in masking_policies.items():
    print(f"Masking policy: {policy_name}")
    process_s3_jsonlines_output_and_save(validation_file_name)
    print("\n")

Masking policy: masked
{"predictions": "Mr. <PATIENT>, <AGE> years-old , born in <CITY>, was transfered to the The <HOSPITAL>. Phone number: <PHONE>. MSW: <MEDICALRECORD> . for his colonic polyps. <DOCTOR> wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood pressure. He said he has cut his alcohol back to 6 pack once a week. <PATIENT> has cut back his cigarettes to one time per week. MR. # <MEDICALRECORD> P:   Follow up with Dr. <DOCTOR> in 3 months. <DOCTOR> P. <DOCTOR>, M.D.  <LOCATION>. PHONE: <PHONE>"}
{"predictions": "Record date : <DATE> , <DOCTOR> , M.D . , Name : <PATIENT>. EMAIL: <EMAIL>. MR # <MEDICALRECORD>  Phone : <PHONE>. Date : <DATE> PCP : <DOCTOR> , <AGE> years-old. A long history of insulin dependent diabetes, type 2; coronary artery disease; chronic renal insufficiency; peripheral vascular disease, also secondary to diabetes; who was originally admitted to an outside hospital for what appeared to be acute paraplegi

In [23]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-de-identify-clinical-deidentificatio-2024-06-13-16-58-45-919


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

